# Imports & Settings


In [2]:
# Imports
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from deep_translator import GoogleTranslator
import re 
from math import isnan
import wandb
import random
from sklearn.ensemble import RandomForestClassifier

import xgboost as xgb
from xgboost import XGBClassifier
# utilities
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_seq_items', None)

In [3]:
# dtype={'type': str} prevents being confused with data type for large data sets
train = pd.read_csv('data/train.csv', index_col='id', dtype={'type': str})
test = pd.read_csv('data/test.csv', index_col='id', dtype={'type': str})
train_translated = pd.read_csv('data/train_translated.csv', dtype={'type': str})
test_translated = pd.read_csv('data/test_translated.csv', index_col='id', dtype={'type': str})
combined_data = pd.read_csv('data/combined_data.csv', index_col='id', dtype={'type': str})
combined_data_translated = pd.read_csv('data/combined_data_translated.csv', index_col='id', dtype={'type': str})
combined_data_fully_translated = pd.read_csv('data/combined_data_fully_translated.csv', index_col='id', dtype={'type': str})
prep = pd.read_csv('data/prep.csv', index_col='id', dtype={'type': str})
test_prep = pd.read_csv('data/test_prepared.csv', index_col='id', dtype={'type': str})
train_prep = pd.read_csv('data/train_prepared.csv', index_col='id', dtype={'type': str})
train_prepROS = pd.read_csv('data/train_prepROS.csv')
train_prepSMOTE = pd.read_csv('data/train_prepSMOTE.csv')


# setup data for classifier

In [4]:
data = train_prepSMOTE.copy()
data.drop(columns=['Unnamed: 0'], inplace=True, axis = 1)

features = data.drop('type', axis=1)
labels = data.type
# at least xgboost cannot deal with string labels
#label_encoder = LabelEncoder()
#label_encoder = label_encoder.fit(labels)
#labels = label_encoder.transform(labels)

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=0)

# setup rf classifier

In [8]:
rfc = RandomForestClassifier(n_estimators=1000,max_depth=2000,max_features=None,min_samples_leaf=2,min_samples_split=2, random_state=42)
rfc.fit(features, labels)

y_pred = rfc.predict(X_test)
probs = rfc.predict_proba(X_test)
val_acc = accuracy_score(y_test, y_pred)

y_pred = rfc.predict(X_train)
probs = rfc.predict_proba(X_train)
train_acc = accuracy_score(y_train, y_pred)

print(train_acc, val_acc)
#0.9577482659448486 0.9236234458259325

# submission (taken from xgboost)

In [ ]:
test_set = test_prep.drop('type', axis=1)
results = rfc.predict(test_set)
submission = pd.DataFrame({'id': test_set.index ,'type': rfc.predict(test_set)})
type_lookup = pd.read_csv('data/type_lookup.csv')
submission = submission.replace(type_lookup.id.to_list(), type_lookup.estonian.to_list())
submission.to_csv('submissions/submission_rf_full_SMOTE.csv', index=False)

# sweep

In [5]:
# Imports
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from deep_translator import GoogleTranslator
import re 
from math import isnan
import wandb
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

# Define sweep config
sweep_configuration = {
    'method': 'bayes',
    'name': 'sweep',
    'metric': {'goal': 'maximize', 'name': 'val_acc'},
    'parameters': 
    {
        'split': {'values': [2, 5, 7, 10, 12, 15, 20]},
        'depth': {'values': [3, 6, 10, 50, 100, 500, 1000, 2000]},
        'leaf': {'values': [2]},
        'estimators': {'values': [100, 200, 350, 500, 1000, 1500, 2000]},
        'features': {'values': [None]},

     }
}

project = 'rf_balanced'
# Initialize sweep by passing in config. (Optional) Provide a name of the project.
sweep_id = wandb.sweep(sweep=sweep_configuration, project=project)

def main():
    run = wandb.init(project=project)

    # note that we define values from `wandb.config` instead 
    # of defining hard values 
    split = wandb.config.split
    depth = wandb.config.depth
    leaf = wandb.config.leaf
    estimators = wandb.config.estimators
    feat = wandb.config.features
    

    # -------------------------- usual training code starts here  -------------------------------------
    
    rfc = RandomForestClassifier(n_estimators=estimators, max_depth=depth, min_samples_leaf=leaf, max_features=feat, min_samples_split=split, random_state=42)
    rfc.fit(X_train, y_train)

    y_pred = rfc.predict(X_test)
    val_acc = accuracy_score(y_test, y_pred)
    
    y_pred = rfc.predict(X_train)
    train_acc = accuracy_score(y_train, y_pred)

    print(train_acc, val_acc)

    # -------------------------- ends here  -------------------------------------
    

    wandb.log({
      'train_acc': train_acc,
      'val_acc': val_acc,
    })

# Start sweep job.
wandb.agent(sweep_id, function=main)

Create sweep with ID: 1apybvok
Sweep URL: https://wandb.ai/museum-item-classification/rf_balanced/sweeps/1apybvok


wandb: Agent Starting Run: 76ap91h4 with config:
wandb: 	depth: 10
wandb: 	estimators: 500
wandb: 	features: None
wandb: 	leaf: 2
wandb: 	split: 5
wandb: Currently logged in as: tillwenke (museum-item-classification). Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


0.40843342919979697 0.392342609038879


train_acc,▁
val_acc,▁
train_acc,0.40843
val_acc,0.39234


wandb: Agent Starting Run: e9l8998q with config:
wandb: 	depth: 2000
wandb: 	estimators: 1000
wandb: 	features: None
wandb: 	leaf: 2
wandb: 	split: 2
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


...

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 6zws8jvf with config:
wandb: 	depth: 2000
wandb: 	estimators: 1500
wandb: 	features: None
wandb: 	leaf: 2
wandb: 	split: 20
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


0.957832854001015 0.9238208012630748


train_acc,▁
val_acc,▁
train_acc,0.95783
val_acc,0.92382


wandb: Agent Starting Run: 0j8i1tay with config:
wandb: 	depth: 2000
wandb: 	estimators: 2000
wandb: 	features: None
wandb: 	leaf: 2
wandb: 	split: 20
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: Ctrl + C detected. Stopping sweep.


# save model

In [5]:
import pickle
pickle.dump(rfc, open('./models/rf/rf_0.7_SMOTE_best_sweep_no_crossval', 'wb'))
loaded_model = pickle.load(open('./models/rf/rf_full_SMOTE', 'rb'))

# submission from model

In [6]:
test_set = test_prep.drop('type', axis=1)

In [7]:
results = rfc.predict(test_set)

In [8]:
submission = pd.DataFrame({'id': test_set.index ,'type': rfc.predict(test_set)})

In [9]:
type_lookup = pd.read_csv('data/type_lookup.csv')

In [10]:
submission = submission.replace(type_lookup.id.to_list(), type_lookup.estonian.to_list())

In [11]:
submission.to_csv('submissions/submission_rf_full_SMOTE.csv', index=False)